In [1]:
import os
import sys

PROJECT_ROOT = \
    os.path.abspath(os.path.join(
        os.path.dirname(""),
        os.pardir))

sys.path.append(PROJECT_ROOT)

import pandas as pd
import matplotlib as plt
import seaborn as sns
import scipy
import numpy as np

In [2]:
# Load in the raw data from the train and test spreadsheets
DATA_RAW_TRAIN = pd.read_csv(PROJECT_ROOT + "/data/raw/train.csv")
DATA_RAW_TEST = pd.read_csv(PROJECT_ROOT + "/data/raw/test.csv")

train = DATA_RAW_TRAIN.copy()
test = DATA_RAW_TEST.copy()

In [3]:
# Check overall statistics
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Cleaning Tasks
1. Convert to numbers:
    - LotShape
    - OverallQual

    - Consider transforming numbers into z-scores, since the numbers themselves are not super important.
2. Remove outliers
    

---
## Standardize column names

In [4]:
# Remap dictionary for original column names
column_remap = {
    "Id": "id",
    "MSSubClass": "ms-subclass",
    "MSZoning": "ms-zoning",
    "LotFrontage": "lot-frontage",
    "LotArea": "lot-area",
    "Street": "roads-street-material",
    "Alley": "roads-alley-material",
    "LotShape": "lot-shape",
    "LandContour": "lot-flatness",
    "Utilities": "utilities-city-provided",
    "LotConfig": "lot-access",
    "LandSlope": "lot-slope",
    "Neighborhood": "location-neighborhood",
    "Condition1": "location-feature-1",
    "Condition2": "location-feature-2",
    "BldgType": "general-dwelling-type",
    "HouseStyle": "construction-style",
    "OverallQual": "ratings-overall-quality",
    "OverallCond": "ratings-overall-condition",
    "YearBuilt": "general-build-year",
    "YearRemodAdd": "general-remodel-year",
    "RoofStyle": "construction-roof-style",
    "RoofMatl":"construction-roof-material",
    "Exterior1st": "construction-exterior-1",
    "Exterior2nd": "construction-exterior-2",
    "MasVnrType": "construction-masonry-type",
    "MasVnrArea": "area-masonry-veneer",
    "ExterQual": "ratings-exterior-quality",
    "ExterCond": "ratings-exterior-condition",
    "Foundation": "construction-foundation-type",
    "BsmtQual": "basement-height",
    "BsmtQual": "ratings-basement-quality",
    "BsmtCond": "ratings-basement-condition",
    "BsmtExposure": "construction-basement-access",
    "BsmtFinType1": "construction-basement-finish-1",
    "BsmtFinSF1": "area-basement-finished-1",
    "BsmtFinType2": "construction-basement-finish-2",
    "BsmtFinSF2": "area-basement-finished-2",
    "BsmtUnfSF": "area-basement-unfinished",
    "TotalBsmtSF": "area-basement-total",
    "Heating": "utilities-heating-type",
    "HeatingQC": "ratings-heating-combined",
    "CentralAir": "utilities-central-air-conditioning",
    "Electrical": "utilities-electrical-wiring-type",
    "1stFlrSF": "area-floor-1",
    "2ndFlrSF": "area-floor-2",
    "LowQualFinSF": "general-total-poor-finish-area",
    "GrLivArea": "area-above-grade-living",
    "BsmtFullBath": "general-number-basement-full-baths",
    "BsmtHalfBath": "general-number-basement-half-baths",
    "FullBath": "general-number-nonbasement-full-baths",
    "HalfBath": "general-number-nonbasement-half-baths",
    "BedroomAbvGr": "general-number-nonbasement-bedrooms",
    "KitchenAbvGr": "general-number-kitchens",
    "KitchenQual": "ratings-kitchen-quality",
    "TotRmsAbvGrd": "general-number-nonbasement-rooms",
    "Functional": "ratings-home-functionality",
    "Fireplaces": "general-number-fireplaces",
    "FireplaceQu": "ratings-fireplace-quality",
    "GarageType": "garage-location",
    "GarageYrBlt": "garage-build-year",
    "GarageFinish": "garage-finish",
    "GarageCars": "general-number-garage-spots",
    "GarageArea": "area-garage",
    "GarageQual": "ratings-garage-quality",
    "GarageCond": "ratings-garage-condition",
    "PavedDrive": "roads-driveway-material",
    "WoodDeckSF": "area-wood-deck",
    "OpenPorchSF": "area-open-porch",
    "EnclosedPorch": "area-closed-porch",
    "3SsnPorch": "area-three-season",
    "ScreenPorch": "area-screen-porch",
    "PoolArea": "area-pool",
    "PoolQC": "ratings-pool-combined",
    "Fence": "lot-fence-material",
    "MiscFeature": "lot-misc-feature",
    "MiscVal": "lot-misc-value",
    "MoSold": "general-month-sold",
    "YrSold": "general-year-sold",
    "SaleType": "general-sale-type",
    "SaleCondition": "general-sale-condition",
    "SalePrice": "sale-price"}

In [5]:
train.columns = \
    pd.Series(train.columns).map(column_remap)
test.columns = \
    pd.Series(test.columns).map(column_remap)

---
## MSSubClass
- Checked for nan
- Remapped categorical

This is a categorical variable falsely masked with numeric codes. All of the listed codes correspond to the codes given in the data dictionary, so I will convert the numbers into the given strings.

In [6]:
subclass_remap = {
        20:	"1-STORY 1946 & NEWER ALL STYLES",
        30:	"1-STORY 1945 & OLDER",
        40:	"1-STORY W/FINISHED ATTIC ALL AGES",
        45:	"1-1/2 STORY - UNFINISHED ALL AGES",
        50:	"1-1/2 STORY FINISHED ALL AGES",
        60:	"2-STORY 1946 & NEWER",
        70:	"2-STORY 1945 & OLDER",
        75:	"2-1/2 STORY ALL AGES",
        80:	"SPLIT OR MULTI-LEVEL",
        85:	"SPLIT FOYER",
        90:	"DUPLEX - ALL STYLES AND AGES",
       120:	"1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
       150:	"1-1/2 STORY PUD - ALL AGES",
       160:	"2-STORY PUD - 1946 & NEWER",
       180:	"PUD - MULTILEVEL - INCL SPLIT LEV/FOYER",
       190:	"2 FAMILY CONVERSION - ALL STYLES AND AGES"}

train["ms-subclass"] = \
    train["ms-subclass"].map(subclass_remap)

test["ms-subclass"] = \
    test["ms-subclass"].map(subclass_remap)

In [7]:
print(train["ms-subclass"].isna().value_counts())
print(test["ms-subclass"].isna().value_counts())

ms-subclass
False    1460
Name: count, dtype: int64
ms-subclass
False    1459
Name: count, dtype: int64


---
## MSZoning
- Checked for nan
    - 10 nan in train set
    - 4 nan in test set
    - All nan replaced with "residential-low-density", the most common option
- Remapped categorical

In [8]:
zoning_remap = {
    "RL": "residential-low-density",
    "RP": "residential-low-density-park",
    "RM": "residential-medium-density",
    "RH": "residential-high-density",
    "A": "agricultural",
    "C": "commercial",
    "FV": "residential-floating-village",
    "I": "industrial"}

train["ms-zoning"] = \
    train["ms-zoning"].map(zoning_remap)
test["ms-zoning"] = \
    test["ms-zoning"].map(zoning_remap)

In [9]:
print(train["ms-zoning"].isna().value_counts())
print(test["ms-zoning"].isna().value_counts())

ms-zoning
False    1450
True       10
Name: count, dtype: int64
ms-zoning
False    1440
True       19
Name: count, dtype: int64


In [10]:
train.loc[train["ms-zoning"].isna(), "ms-zoning"] = "residential-low-density"
test.loc[test["ms-zoning"].isna(), "ms-zoning"] = "residential-low-density"

In [11]:
print(train["ms-zoning"].isna().value_counts())
print(test["ms-zoning"].isna().value_counts())

ms-zoning
False    1460
Name: count, dtype: int64
ms-zoning
False    1459
Name: count, dtype: int64


---
## Lot Frontage
- Checked for nan
    - 259 nan values (17%) in train set
    - 227 nan values (16%) in test set
- Worked on a proxy for lot frontage:
    - Related to most lot features:
        - lot-area, lot-shape, lot-access
    - lot-frontage varies with lot-shape
        - median lot-frontage for...
            - lot-shape Reg ==> 67
            - lot-shape IR1 ==> 74
            - lot-shape IR2 ==> 57.5
            - lot-shape IR3 ==> 150
    - set missing values to the median values for their lot shape

In [12]:
print(train["lot-frontage"].isna().value_counts())
print(test["lot-frontage"].isna().value_counts())

lot-frontage
False    1201
True      259
Name: count, dtype: int64
lot-frontage
False    1232
True      227
Name: count, dtype: int64


In [13]:
reg_lots = train.loc[train.loc[:, "lot-shape"] == "Reg", :]
ir1_lots = train.loc[train.loc[:, "lot-shape"] == "IR1", :]
ir2_lots = train.loc[train.loc[:, "lot-shape"] == "IR2", :]
ir3_lots = train.loc[train.loc[:, "lot-shape"] == "IR3", :]

In [14]:
reg_lots_no_nan = reg_lots.loc[~reg_lots["lot-frontage"].isna(), :]
ir1_lots_no_nan = ir1_lots.loc[~ir1_lots["lot-frontage"].isna(), :]
ir2_lots_no_nan = ir2_lots.loc[~ir2_lots["lot-frontage"].isna(), :]
ir3_lots_no_nan = ir3_lots.loc[~ir3_lots["lot-frontage"].isna(), :]

In [15]:
reg_median = np.median(reg_lots_no_nan["lot-frontage"])
ir1_median = np.median(ir1_lots_no_nan["lot-frontage"])
ir2_median = np.median(ir2_lots_no_nan["lot-frontage"])
ir3_median = np.median(ir3_lots_no_nan["lot-frontage"])

In [16]:
train.loc[((train.loc[:, "lot-shape"] == "Reg") 
           & (train["lot-frontage"].isna())), "lot-frontage"] = \
    reg_median

train.loc[((train.loc[:, "lot-shape"] == "IR1") 
           & (train["lot-frontage"].isna())), "lot-frontage"] = \
    ir1_median

train.loc[((train.loc[:, "lot-shape"] == "IR2") 
           & (train["lot-frontage"].isna())), "lot-frontage"] = \
    ir2_median

train.loc[((train.loc[:, "lot-shape"] == "IR3") 
           & (train["lot-frontage"].isna())), "lot-frontage"] = \
    ir3_median

In [17]:
test.loc[((test.loc[:, "lot-shape"] == "Reg") 
           & (test["lot-frontage"].isna())), "lot-frontage"] = \
    reg_median

test.loc[((test.loc[:, "lot-shape"] == "IR1") 
           & (test["lot-frontage"].isna())), "lot-frontage"] = \
    ir1_median

test.loc[((test.loc[:, "lot-shape"] == "IR2") 
           & (test["lot-frontage"].isna())), "lot-frontage"] = \
    ir2_median

test.loc[((test.loc[:, "lot-shape"] == "IR3") 
           & (test["lot-frontage"].isna())), "lot-frontage"] = \
    ir3_median

---
## Lot Area
- Checked for nan
    - No nan in either train or test

In [18]:
print(train["lot-area"].isna().value_counts())
print(test["lot-area"].isna().value_counts())

lot-area
False    1460
Name: count, dtype: int64
lot-area
False    1459
Name: count, dtype: int64


---
## Street and Alleys
- Remapped categoricals

In [19]:
street_remap = {
    "Grvl": "gravel",
    "Pave": "paved",
    np.nan: "none"}

In [20]:
train["roads-street-material"] = \
      train["roads-street-material"].map(street_remap)
test["roads-street-material"] = \
      test["roads-street-material"].map(street_remap)

train["roads-alley-material"] = \
      train["roads-alley-material"].map(street_remap)
test["roads-alley-material"] = \
      test["roads-alley-material"].map(street_remap)

---
## Lot Shape
- Checked for nan
- Renamed categoricals

In [21]:
lotshape_map = {
    "Reg": "regular",
    "IR1": "slightly-irregular",
    "IR2": "moderately-irregular",
    "IR3": "irregular"}

In [22]:
train["lot-shape"] = \
    train["lot-shape"].map(lotshape_map)
test["lot-shape"] = \
    test["lot-shape"].map(lotshape_map)

---
## Land Countour
- Checked for nan
- Renamed categoricals

In [23]:
contour_map = {
    "Lvl": "level",
    "Bnk": "slope-from-street",
    "HLS": "slope-along-street",
    "Low": "below-grade"}

In [24]:
train["lot-flatness"] = \
    train["lot-flatness"].map(contour_map)

test["lot-flatness"] = \
    test["lot-flatness"].map(contour_map)

---
## Utilities
- Checked for nan
    - 2 nan found in test dataset
        - Set nan to "AllPub" (all public utilities)

- Remapped categoricals

In [25]:
utilities_map = {
    "AllPub": "electricity-gas-water-sewer",
    "NoSewr": "electricity-gas-water",
    "NoSeWa": "electricity-gas",
    "ELO": "electricity",
    np.nan: "electricity-gas-water-sewer"}

In [26]:
train["utilities-city-provided"] = \
    train["utilities-city-provided"].map(utilities_map)
test["utilities-city-provided"] = \
    test["utilities-city-provided"].map(utilities_map)

---
## Lot Config
- Checked for nan
- remapped categoricals

In [27]:
access_map = {
    "Inside": "inside",
    "Corner": "corner",
    "CulDSac": "culdsac",
    "FR2": "double-frontage",
    "FR3": "triple-frontage"}

In [28]:
train["lot-access"] = \
    train["lot-access"].map(access_map)
test["lot-access"] = \
    test["lot-access"].map(access_map)

---
## Land Slope
- Checked for nan
- remapped categoricals

In [29]:
slope_map = {
    "Gtl": "gentle-slope",
    "Mod": "moderate-slope",
    "Sev": "severe-slope"}

In [30]:
train["lot-slope"] = \
    train["lot-slope"].map(slope_map)
test["lot-slope"] = \
    test["lot-slope"].map(slope_map)

---
## Neighborhood
- checked for nan
- remapped

In [31]:

neighborhood_map = {
    "Blmngtn": "bloomington-heights",
    "Blueste": "bluestem",
    "BrDale": "briardale",
    "BrkSide": "brookside",
    "ClearCr": "clear-creek",
    "CollgCr": "college-creek",
    "Crawfor": "crawford",
    "Edwards": "edwards",
    "Gilbert": "gilberts",
    "IDOTRR": "iowa-dot-rail-road",
    "MeadowV": "meadow-village",
    "Mitchel": "mitchell",
    "NAmes": "north-ames",
    "NoRidge": "northridge",
    "NPkVill": "northpart-villa",
    "NridgHt": "northridge-heights",
    "NWAmes": "northwest-ames",
    "OldTown": "old-town",
    "SWISU": "south-and-west-iowa-state",
    "Sawyer": "sawyer",
    "SawyerW": "sawyer-west",
    "Somerst": "somerset",
    "StoneBr": "stone-brook",
    "Timber": "timberland",
    "Veenker": "veenker"}

In [32]:
train["location-neighborhood"] = \
    train["location-neighborhood"].map(neighborhood_map)
test["location-neighborhood"] = \
    test["location-neighborhood"].map(neighborhood_map)

---
## Condition1 and Condition2
- checked for nan
- decided against remapping

---
---
Following columns still contained null values. They were managed as described below:
- Unchanged, as NA indicated absence of a feature:
    - 'ratings-basement-quality'
    - 'ratings-basement-condition',
    - 'construction-basement-access'
    - 'construction-basement-finish-1'
    - 'construction-basement-finish-2'
    - 'ratings-fireplace-quality'
    - 'garage-location'
    - 'garage-build-year'
    - 'garage-finish'
    - 'ratings-garage-quality'
    - 'ratings-garage-condition',
    - 'lot-fence-material'
    - 'lot-misc-feature'
    - 'ratings-pool-combined'



- 'utilities-electrical-wiring-type'
    - Only one item with this found. Filled this value with "SBrkr", since it was built in 2006

In [33]:
train.loc[train["utilities-electrical-wiring-type"].isna(), "utilities-electrical-wiring-type"] = "SBrkr"

In [34]:
masonry_type_map = {
    np.nan: "non-masonry",
    "BrkFace": "brick-face",
    "Stone": "stone",
    "BrkCmn": "brick-common"}

train["construction-masonry-type"] = \
    train["construction-masonry-type"].map(masonry_type_map)
test["construction-masonry-type"] = \
    test["construction-masonry-type"].map(masonry_type_map)

In [35]:
train["area-masonry-veneer"] = \
    train["area-masonry-veneer"].fillna(0)
test["area-masonry-veneer"] = \
    test["area-masonry-veneer"].fillna(0)

In [36]:
test["area-masonry-veneer"].value_counts(dropna=False)

area-masonry-veneer
0.0       892
176.0      10
144.0       9
120.0       8
216.0       8
         ... 
647.0       1
1290.0      1
495.0       1
292.0       1
382.0       1
Name: count, Length: 303, dtype: int64

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     1460 non-null   int64  
 1   ms-subclass                            1460 non-null   object 
 2   ms-zoning                              1460 non-null   object 
 3   lot-frontage                           1460 non-null   float64
 4   lot-area                               1460 non-null   int64  
 5   roads-street-material                  1460 non-null   object 
 6   roads-alley-material                   1460 non-null   object 
 7   lot-shape                              1460 non-null   object 
 8   lot-flatness                           1460 non-null   object 
 9   utilities-city-provided                1460 non-null   object 
 10  lot-access                             1460 non-null   object 
 11  lot-

In [38]:
# Convert columns that do not need to be floats into ints
float_columns = ["lot-frontage", "area-masonry-veneer"]

for column_name in float_columns:
    train[column_name] = train[column_name].astype(int)
    test[column_name] = test[column_name].astype(int)

In [39]:
train.to_csv(PROJECT_ROOT + "/data/interim/train-cleaned.csv")
test.to_csv(PROJECT_ROOT + "/data/interim/test-cleaned.csv")